In [1]:
%load_ext watermark
%watermark

ModuleNotFoundError: No module named 'watermark'

In [3]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [5]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [9]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
print(file_path)

time       = []
T_aux_avg  = []
r2_v2_rlim      = []

for i,j in enumerate(file_path):
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])

('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC03_RF03.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC04_RF04.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC05_RF05.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC06_RF06.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_tem

In [10]:
def tanh_plus(x,K,theta,x0,a,b):
    return K*(1+tanh(theta*(x-x0))) + a*(x)+b
def tanh_plus_to10(x,K,theta,x0,a,b):
    return 10**(K*(1+tanh(theta*(x-x0))) + a*(x)+b)
def gen_logi(x,A,K,C,B,M,nu):
    return A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu)
def gen_logi_to10(x,A,K,C,B,M,nu):
    return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )
def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B
def func3(t, t0, A, B,  C):
    return A*(t-t0) / (C+(t-t0)**2) + B
def func4(t, t0, A, B, C):
    return A*(t-t0) / (C+abs(t-t0)) + B
def func5(t, t0, A, B, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B

In [11]:
# OK pour la temperature
to_plot = 1
end_fit = 125500

print(file_path[to_plot])
# test plot
index_max = 1900
figure(1,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
plot(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')

p0 = [1.8, 60, 63.45, 0.002,4e-3] # 124850
popt, pcov = curve_fit(tanh_plus_to10,
                       time[to_plot][:end_fit]*1e3, T_aux_avg[to_plot][:end_fit],
                       p0, maxfev= 50000)
print(popt)
plot(time[to_plot][:end_fit]*1e3,tanh_plus_to10(time[to_plot][:end_fit]*1e3,*popt),color='k')

plot(time[to_plot][end_fit:]*1e3,tanh_plus_to10(time[to_plot][end_fit:]*1e3,
                                            1635.3,
                                            14.075,
                                            63.132,
                                            0.87,
                                            -3324.94),
     color='g',ls=':')


p0 = [5e-3,500,1,2,65,2] # 124850
popt, pcov = curve_fit(gen_logi_to10,
                       time[to_plot][:end_fit]*1e3, T_aux_avg[to_plot][:end_fit],
                       maxfev= 50000)
plot(time[to_plot][:end_fit]*1e3,gen_logi_to10(time[to_plot][:end_fit]*1e3,*popt),color='xkcd:azul')

vlines(time[to_plot][end_fit]*1e3,0,50)
#b, a = signal.butter(8, 0.125)
#y = signal.filtfilt(b, a, T_aux_avg[to_plot], padlen=150)
#semilogy(time[to_plot]*1e3,y,color='b')
xlabel('time [ms]')
ylabel('T [K]')
#xlim(61,64)
#ylim(-1,20)
grid()
tight_layout()

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
[ 1.63532814e+03  1.40758340e+01  6.31322873e+01  8.72241875e-01
 -3.32494371e+03]


/tmp/ipykernel_90434/3493093275.py:8: RuntimeWarning: overflow encountered in exp
  return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )
/tmp/ipykernel_90434/3493093275.py:8: RuntimeWarning: overflow encountered in power
  return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )
/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [38]:
to_fit = 3
end_fit = 7000
x  = time[to_fit]
y  = T_aux_avg[to_fit]
figure('testfitinlog',clear='True')
semilogy(x*1e3, y    )
vlines(x[end_fit]*1e3,1e-2,300,color='k')

p0 = [2.5e-3,2,0,0]
popt, pcov = curve_fit(func4, x[:end_fit], log10(y[:end_fit]), p0,  maxfev= 500000)
y_fit = func4(x[:end_fit],*popt)
print(popt)
semilogy(x[:end_fit]*1e3,10**y_fit,color='g')
#semilogy(x*1e3,10**func4(x,3.81e-3,1,0,0),color='xkcd:orange')

p0 = [2.5e-3,2,-2,0,1]
popt, pcov = curve_fit(func5, x[:end_fit], log10(y[:end_fit]), p0, maxfev= 500000)
print(popt)
y_fit = func5(x[:end_fit],*popt)
semilogy(x[:end_fit]*1e3,10**y_fit,color='r')

[ 2.52957473e-03  1.93087463e+00 -2.81974191e-01  3.29060510e-05]


/tmp/ipykernel_90434/3493093275.py:16: RuntimeWarning: overflow encountered in power
  return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B
/tmp/ipykernel_90434/3493093275.py:16: RuntimeWarning: divide by zero encountered in true_divide
  return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B


[ 0.00252949  2.48851053 -0.29112619  0.00999985  0.32766011]


In [129]:
# OK pour la temperature
x = linspace(-10,10,1000)

print(file_path[to_plot])
# test plot
index_max = 1900
figure(1111,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
plot(x,tanh_plus(x,1,1,0,0.1,0),color='r')
#b, a = signal.butter(8, 0.125)
#y = signal.filtfilt(b, a, T_aux_avg[to_plot], padlen=150)
#semilogy(time[to_plot]*1e3,y,color='b')
xlabel('time [ms]')
ylabel('T [K]')
# xlim(1,1.1)
grid()
tight_layout()

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
